In [143]:
# Import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re, string
import math
from collections import Counter

In [144]:
path = "C:/Users/Bertha/Documents/SJSU/5_Spring2024/DATA_240-Data_Mining_Analytics/DATA240_Project/DATA240/data/Supply_Chain_Shipment_Pricing_Dataset_20240302.csv"
data = pd.read_csv(path, index_col="id") # ID col is treated as the index
data

,project code,pq #,po / so #,asn/dn #,country,managed by,fulfill via,vendor inco term,shipment mode,pq first sent to client date,...,unit of measure (per pack),line item quantity,line item value,pack price,unit price,manufacturing site,first line designation,weight (kilograms),freight cost (usd),line item insurance (usd)
id,,,,,,,,,,,,,,,,,,,,,
1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,...,30,19,551.00,29.00,0.97,Ranbaxy Fine Chemicals LTD,True,13,780.34,NaN
3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,...,240,1000,6200.00,6.20,0.03,"Aurobindo Unit III, India",True,358,4521.5,NaN
4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,Pre-PQ Process,...,100,500,40000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,True,171,1653.78,NaN
15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,...,60,31920,127360.80,3.99,0.07,"Ranbaxy, Paonta Shahib, India",True,1855,16007.06,NaN
16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,...,60,38000,121600.00,3.20,0.05,"Aurobindo Unit III, India",True,7590,45450.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,10/16/2014,...,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",False,See DN-4307 (ID#:83920),See DN-4307 (ID#:83920),705.79
86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,10/24/2014,...,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,False,See DN-4313 (ID#:83921),See DN-4313 (ID#:83921),161.71
86821,110-ZM-T30,FPQ-14784,SO-49600,DN-4316,Zambia,PMO - US,From RDC,N/A - From RDC,Truck,8/12/2014,...,30,514526,5140114.74,9.99,0.33,Cipla Ltd A-42 MIDC Mahar. IN,False,Weight Captured Separately,Freight Included in Commodity Cost,5284.04


# Data Cleaning

In [145]:
# for each col replace special characters
newcols = [i.strip().replace(" ","_").replace("#","num").replace("/","_").replace("(","").replace(")","") for i in data.columns]
data.columns = newcols

In [146]:
data.shape

(10324, 32)

In [147]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10324 entries, 1 to 86823
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   project_code                  10324 non-null  object 
 1   pq_num                        10324 non-null  object 
 2   po___so_num                   10324 non-null  object 
 3   asn_dn_num                    10324 non-null  object 
 4   country                       10324 non-null  object 
 5   managed_by                    10324 non-null  object 
 6   fulfill_via                   10324 non-null  object 
 7   vendor_inco_term              10324 non-null  object 
 8   shipment_mode                 9964 non-null   object 
 9   pq_first_sent_to_client_date  10324 non-null  object 
 10  po_sent_to_vendor_date        10324 non-null  object 
 11  scheduled_delivery_date       10324 non-null  object 
 12  delivered_to_client_date      10324 non-null  object 
 13  delive

In [148]:
# cols have nulls: shipment_mode, dosage, and line_item_insurance_usd
data.isnull().sum()

project_code                       0
pq_num                             0
po___so_num                        0
asn_dn_num                         0
country                            0
managed_by                         0
fulfill_via                        0
vendor_inco_term                   0
shipment_mode                    360
pq_first_sent_to_client_date       0
po_sent_to_vendor_date             0
scheduled_delivery_date            0
delivered_to_client_date           0
delivery_recorded_date             0
product_group                      0
sub_classification                 0
vendor                             0
item_description                   0
molecule_test_type                 0
brand                              0
dosage                          1736
dosage_form                        0
unit_of_measure_per_pack           0
line_item_quantity                 0
line_item_value                    0
pack_price                         0
unit_price                         0
m

In [149]:
# check duplicates
data.duplicated().sum()

4

In [150]:
# 4 duplicates (8 total rows)
data[data.duplicated(keep=False)]

,project_code,pq_num,po___so_num,asn_dn_num,country,managed_by,fulfill_via,vendor_inco_term,shipment_mode,pq_first_sent_to_client_date,...,unit_of_measure_per_pack,line_item_quantity,line_item_value,pack_price,unit_price,manufacturing_site,first_line_designation,weight_kilograms,freight_cost_usd,line_item_insurance_usd
id,,,,,,,,,,,,,,,,,,,,,
10692,125-HT-T01,Pre-PQ Process,SO-1291,DN-686,Haiti,PMO - US,From RDC,N/A - From RDC,Air,Pre-PQ Process,...,100,10,720.0,72.00,0.72,Inverness Japan,False,See DN-686 (ID#:10691),See DN-686 (ID#:10691),1.15
11246,125-HT-T01,Pre-PQ Process,SO-1291,DN-686,Haiti,PMO - US,From RDC,N/A - From RDC,Air,Pre-PQ Process,...,100,10,720.0,72.00,0.72,Inverness Japan,False,See DN-686 (ID#:10691),See DN-686 (ID#:10691),1.15
11882,100-ZW-T01,Pre-PQ Process,SO-716,DN-770,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Air,Pre-PQ Process,...,30,1860,23101.2,12.42,0.41,"Aurobindo Unit III, India",False,See DN-770 (ID#:10959),See DN-770 (ID#:10959),36.96
12146,100-ZW-T01,Pre-PQ Process,SO-710,DN-488,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Air,Pre-PQ Process,...,30,1860,23101.2,12.42,0.41,"Cipla, Goa, India",False,See DN-488 (ID#:10652),See DN-488 (ID#:10652),36.96
12147,100-ZW-T01,Pre-PQ Process,SO-710,DN-488,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Air,Pre-PQ Process,...,30,1860,23101.2,12.42,0.41,"Cipla, Goa, India",False,See DN-488 (ID#:10652),See DN-488 (ID#:10652),36.96
12792,100-ZW-T01,Pre-PQ Process,SO-716,DN-770,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Air,Pre-PQ Process,...,30,1860,23101.2,12.42,0.41,"Aurobindo Unit III, India",False,See DN-770 (ID#:10959),See DN-770 (ID#:10959),36.96
43250,105-SS-T30,FPQ-12623,SCMS-200920,ASN-21751,South Sudan,PMO - US,Direct Drop,EXW,Air,7/18/2013,...,20,25,800.0,32.00,1.60,"Trinity Biotech, Plc",False,See ASN-21751 (ID#:13968),See ASN-21751 (ID#:13968),0.99
68128,105-SS-T30,FPQ-12623,SCMS-200920,ASN-21751,South Sudan,PMO - US,Direct Drop,EXW,Air,7/18/2013,...,20,25,800.0,32.00,1.60,"Trinity Biotech, Plc",False,See ASN-21751 (ID#:13968),See ASN-21751 (ID#:13968),0.99


In [151]:
# drop duplicate rows
data = data.drop_duplicates()

In [152]:
# check dataframe
data[data.duplicated(keep=False)]

,project_code,pq_num,po___so_num,asn_dn_num,country,managed_by,fulfill_via,vendor_inco_term,shipment_mode,pq_first_sent_to_client_date,...,unit_of_measure_per_pack,line_item_quantity,line_item_value,pack_price,unit_price,manufacturing_site,first_line_designation,weight_kilograms,freight_cost_usd,line_item_insurance_usd
id,,,,,,,,,,,,,,,,,,,,,


In [153]:
# after dropping duplicates, check 
data.duplicated().sum()

0

In [154]:
data.shape # Before: 10324 rows
           # After: 10320 rows

(10320, 32)

In [155]:
data

,project_code,pq_num,po___so_num,asn_dn_num,country,managed_by,fulfill_via,vendor_inco_term,shipment_mode,pq_first_sent_to_client_date,...,unit_of_measure_per_pack,line_item_quantity,line_item_value,pack_price,unit_price,manufacturing_site,first_line_designation,weight_kilograms,freight_cost_usd,line_item_insurance_usd
id,,,,,,,,,,,,,,,,,,,,,
1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,...,30,19,551.00,29.00,0.97,Ranbaxy Fine Chemicals LTD,True,13,780.34,NaN
3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,...,240,1000,6200.00,6.20,0.03,"Aurobindo Unit III, India",True,358,4521.5,NaN
4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,Pre-PQ Process,...,100,500,40000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,True,171,1653.78,NaN
15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,...,60,31920,127360.80,3.99,0.07,"Ranbaxy, Paonta Shahib, India",True,1855,16007.06,NaN
16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,...,60,38000,121600.00,3.20,0.05,"Aurobindo Unit III, India",True,7590,45450.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,10/16/2014,...,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",False,See DN-4307 (ID#:83920),See DN-4307 (ID#:83920),705.79
86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,10/24/2014,...,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,False,See DN-4313 (ID#:83921),See DN-4313 (ID#:83921),161.71
86821,110-ZM-T30,FPQ-14784,SO-49600,DN-4316,Zambia,PMO - US,From RDC,N/A - From RDC,Truck,8/12/2014,...,30,514526,5140114.74,9.99,0.33,Cipla Ltd A-42 MIDC Mahar. IN,False,Weight Captured Separately,Freight Included in Commodity Cost,5284.04


# Clean categorical attributes with noisy text fields

In [156]:
# cols
data[['item_description', 'molecule_test_type', 'manufacturing_site', 'weight_kilograms', 'freight_cost_usd']].head()

,item_description,molecule_test_type,manufacturing_site,weight_kilograms,freight_cost_usd
id,,,,,
1,"HIV, Reveal G3 Rapid HIV-1 Antibody Test, 30 T...","HIV, Reveal G3 Rapid HIV-1 Antibody Test",Ranbaxy Fine Chemicals LTD,13,780.34
3,"Nevirapine 10mg/ml, oral suspension, Bottle, 2...",Nevirapine,"Aurobindo Unit III, India",358,4521.5
4,"HIV 1/2, Determine Complete HIV Kit, 100 Tests","HIV 1/2, Determine Complete HIV Kit",ABBVIE GmbH & Co.KG Wiesbaden,171,1653.78
15,"Lamivudine 150mg, tablets, 60 Tabs",Lamivudine,"Ranbaxy, Paonta Shahib, India",1855,16007.06
16,"Stavudine 30mg, capsules, 60 Caps",Stavudine,"Aurobindo Unit III, India",7590,45450.08


In [157]:
# converts previous index to col again after identifying duplicates
data = data.reset_index()
data

,id,project_code,pq_num,po___so_num,asn_dn_num,country,managed_by,fulfill_via,vendor_inco_term,shipment_mode,...,unit_of_measure_per_pack,line_item_quantity,line_item_value,pack_price,unit_price,manufacturing_site,first_line_designation,weight_kilograms,freight_cost_usd,line_item_insurance_usd
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.00,29.00,0.97,Ranbaxy Fine Chemicals LTD,True,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.00,6.20,0.03,"Aurobindo Unit III, India",True,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,True,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.80,3.99,0.07,"Ranbaxy, Paonta Shahib, India",True,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.00,3.20,0.05,"Aurobindo Unit III, India",True,7590,45450.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10315,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",False,See DN-4307 (ID#:83920),See DN-4307 (ID#:83920),705.79
10316,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,...,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,False,See DN-4313 (ID#:83921),See DN-4313 (ID#:83921),161.71
10317,86821,110-ZM-T30,FPQ-14784,SO-49600,DN-4316,Zambia,PMO - US,From RDC,N/A - From RDC,Truck,...,30,514526,5140114.74,9.99,0.33,Cipla Ltd A-42 MIDC Mahar. IN,False,Weight Captured Separately,Freight Included in Commodity Cost,5284.04
10318,86822,200-ZW-T30,FPQ-16523,SO-51680,DN-4334,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,17465,113871.80,6.52,0.11,Mylan (formerly Matrix) Nashik,True,1392,Freight Included in Commodity Cost,134.03


Use index col (id) to convert weight and freight

## weight_kilograms & freight_cost_usd

In [158]:
# before conversion 
data[['id', 'freight_cost_usd', 'weight_kilograms']].tail()

,id,freight_cost_usd,weight_kilograms
10315,86818,See DN-4307 (ID#:83920),See DN-4307 (ID#:83920)
10316,86819,See DN-4313 (ID#:83921),See DN-4313 (ID#:83921)
10317,86821,Freight Included in Commodity Cost,Weight Captured Separately
10318,86822,Freight Included in Commodity Cost,1392
10319,86823,Freight Included in Commodity Cost,Weight Captured Separately


In [159]:
# match any digit in object for id_num column
dic = {"id_num": ":\d*"} # ex: ID#:83921

def convert_num_freight(fcu):
    # regex function to search for pattern defined in dic 
    found_m = re.search(dic['id_num'], fcu, re.IGNORECASE) # ignore upper/lowercase
    # if match is found (ID#:83921) it only returns the digits
    if found_m:
        # remove colon from the element stored in found_m 
        ident = found_m.group(0).replace(':','')
        # query data on id col
        f = data.query("id == "+ident)
        return f['freight_cost_usd'].item()
    else: # return original element if no match found like 80111
        return fcu
    
def convert_num_weight(w):
    found_m = re.search(dic['id_num'], w, re.IGNORECASE)
    if found_m:
        ident = found_m.group(0).replace(':','')
        f = data.query("id == "+ident)
        return f['weight_kilograms'].item()
    else:
        return w

In [160]:
# this takes some time to load 10-30 seconds
# apply defined function to each element of the freight and weight columns
data['freight_cost_usd'] = data['freight_cost_usd'].apply(convert_num_freight)
data['weight_kilograms'] = data['weight_kilograms'].apply(convert_num_weight)

In [161]:
# after conversion
# when element in weight_kilograms col says: "See DN-4307 (ID#:83920)"
# row 10315 (id 86818) matches id 83920 
data[['id', 'freight_cost_usd', 'weight_kilograms']].tail(5)

,id,freight_cost_usd,weight_kilograms
10315,86818,46111.55,25880
10316,86819,14734.92,4426
10317,86821,Freight Included in Commodity Cost,Weight Captured Separately
10318,86822,Freight Included in Commodity Cost,1392
10319,86823,Freight Included in Commodity Cost,Weight Captured Separately


In [162]:
# check row 7933 with id 83920 to compare with row 10315
data[['id', 'freight_cost_usd', 'weight_kilograms']][data['id']==83920]

,id,freight_cost_usd,weight_kilograms
7933,83920,46111.55,25880


After conversion, we've referenced their corresponding id and obtains the freight_cost_usd and weight_kilograms. For example, row 10315 previously had elements of "See DN-4307 (ID#:83920)" and "See DN-4307 (ID#:83920)" for both freight and weight columns. After conversion, we see it has referenced id 83920

Now we will look for the indexes with these objects in the weight_kilograms and freight_cost_usd columns, which we are trying to convert to numeric/float data type: 

'Weight Captured Separately', 'Freight Included in Commodity Cost' & 'Invoiced Separately' elements only

In [163]:
print("Number of 'Freight Included in Commodity Cost' elements:", len(data[data.freight_cost_usd == 'Freight Included in Commodity Cost']))
print("Number of 'Invoiced Separately' elements:", len(data[data.freight_cost_usd == 'Invoiced Separately']))
print("Number of 'Weight Captured Separately' elements:", len(data[data.weight_kilograms == 'Weight Captured Separately']))

Number of 'Freight Included in Commodity Cost' elements: 1442
Number of 'Invoiced Separately' elements: 344
Number of 'Weight Captured Separately' elements: 1633


In [164]:
# in weight_kilograms col find 'Weight Captured Separately' elements
# store boolean values as a list
w_bool = (data['weight_kilograms'] == 'Weight Captured Separately')
w_bool

0        False
1        False
2        False
3        False
4        False
         ...  
10315    False
10316    False
10317     True
10318    False
10319     True
Name: weight_kilograms, Length: 10320, dtype: bool

In [165]:
# convert to list
w_bool_list = w_bool.tolist()
w_bool_list

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True

In [166]:
# get index of each element in list
w_idx = data.index[w_bool_list]
w_idx

Index([   12,    15,    16,    32,    36,    37,    38,    41,    43,    48,
       ...
       10168, 10169, 10172, 10178, 10184, 10185, 10187, 10296, 10317, 10319],
      dtype='int64', length=1633)

In [167]:
# put altogether into a single line of code
# finds list of indices where weight_kilograms col contains 'Weight Captured Separately' value
w_index = data.index[data['weight_kilograms'] == 'Weight Captured Separately'].tolist()
w_index

[12,
 15,
 16,
 32,
 36,
 37,
 38,
 41,
 43,
 48,
 49,
 51,
 53,
 54,
 56,
 57,
 58,
 62,
 66,
 70,
 73,
 77,
 78,
 79,
 80,
 82,
 83,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 114,
 115,
 116,
 128,
 129,
 130,
 144,
 150,
 159,
 160,
 165,
 167,
 175,
 182,
 183,
 184,
 185,
 189,
 190,
 191,
 192,
 195,
 196,
 197,
 198,
 204,
 205,
 206,
 207,
 220,
 221,
 224,
 226,
 227,
 228,
 229,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 250,
 251,
 252,
 254,
 255,
 256,
 259,
 270,
 271,
 272,
 302,
 306,
 307,
 308,
 310,
 311,
 312,
 313,
 314,
 315,
 320,
 321,
 322,
 327,
 328,
 333,
 335,
 336,
 337,
 338,
 342,
 348,
 349,
 350,
 351,
 362,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 391,
 392,
 393,
 394,
 395,
 397,
 401,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 424,
 425,
 428,
 430,
 431,
 436,
 437,
 438,
 440,
 446,
 447,
 448,
 449,
 455,
 461,
 463,
 465,
 471,
 478,
 47

In [168]:
# repeat for freight_cost_usd col
fcu_index = data.index[(data['freight_cost_usd'] == 'Freight Included in Commodity Cost') | (data['freight_cost_usd'] == 'Invoiced Separately')].tolist()

# 'Freight Included in Commodity Cost' elements only
fcu_included_index = data.index[(data['freight_cost_usd'] == 'Freight Included in Commodity Cost')].tolist()

# 'Invoiced Separately' elements only
fcu_inv_sep_index = data.index[(data['freight_cost_usd'] == 'Invoiced Separately')].tolist()

In [169]:
print("weight_kilograms col:")
print("'Weight Captured Separately' elements:", len(w_index))
print()
print()

print("freight_cost_usd col:", "total number of indices with either of these elements:", len(fcu_index))
print()
print("'Freight Included in Commodity Cost' elements:", len(fcu_included_index))
print()
print("'Invoiced Separately' elements:", len(fcu_inv_sep_index))

weight_kilograms col:
'Weight Captured Separately' elements: 1633


freight_cost_usd col: total number of indices with either of these elements: 1786

'Freight Included in Commodity Cost' elements: 1442

'Invoiced Separately' elements: 344


------------------------
When freight cost is invoiced separately it means the cost of freight (the transportation of bulk goods via truck, plane, etc.) is not included in the total cost of being shipped. Thus, it's billed separately as freight costs vary depending on location, if it's transported domestically or internationally, etc. Some customers may also prefer to have their goods and transportation of the goods be billed separately for financial purposes. Or customers may want to use a different shipping service rather than have it be delivered to them directly, such as having a middleman to handle the process like Regional Distribution Centers. 

When the weight is captured separately, that means the actual weight of the commodities is separated from the weight of package used to transport them. Thus, the actual weight and weight of package are recorded separately in order to accurately determine shipment costs. This ensures accurate billing based on pure weights of the goods themselves.

Since 'line item quantity x pack price = line item value' & the line item value is the 'total value of commodity per line item' according to the data source information, we can set freight cost to 0 because freight cost is already included in the line item value. Thus, for elements with 'Freight Included in Commodity Cost', we set to 0. Also, important to note when we can see when 'Freight Included in Commodity Cost', the 'line item insurance (usd)' are all blank.

Then, we can impute 1633 'Weight Captured Separately' elements and 344 'Invoiced Separately' elements using the median since they are continuous data types. Assume missing value to determine subgroup-specific median. Then, impute those elements of missing values with the median obtained from subgroup

-------------------------

* Use mode for categorical or discrete
* Use mean or median for continuous data

### Weight Captured Separately

In [170]:
# convert 1633 'Weight Captured Separately' elements to nan
data['weight_kilograms'] = data['weight_kilograms'].replace('Weight Captured Separately', np.nan)
data['weight_kilograms']

0           13
1          358
2          171
3         1855
4         7590
         ...  
10315    25880
10316     4426
10317      NaN
10318     1392
10319      NaN
Name: weight_kilograms, Length: 10320, dtype: object

In [171]:
'''
# convert to float dtype
data['weight_kilograms'] = pd.to_numeric(data['weight_kilograms'], errors='coerce')
data['weight_kilograms']

# check nans to confirm
print(data['weight_kilograms'].isna().sum())
'''

"\n# convert to float dtype\ndata['weight_kilograms'] = pd.to_numeric(data['weight_kilograms'], errors='coerce')\ndata['weight_kilograms']\n\n# check nans to confirm\nprint(data['weight_kilograms'].isna().sum())\n"

In [172]:
# impute missing values using median for continous data
weight_median = data['weight_kilograms'].median()
weight_median

1303.0

In [173]:
data['weight_kilograms'] = data['weight_kilograms'].fillna(weight_median)
data['weight_kilograms']

0            13
1           358
2           171
3          1855
4          7590
          ...  
10315     25880
10316      4426
10317    1303.0
10318      1392
10319    1303.0
Name: weight_kilograms, Length: 10320, dtype: object

We see the last element converted from NaN to the median value of weight_kilograms 1303.0

### Freight Included in Commodity Cost

In [174]:
data['freight_cost_usd'] = data['freight_cost_usd'].replace('Freight Included in Commodity Cost', 0)
data['freight_cost_usd'].tail()

10315    46111.55
10316    14734.92
10317           0
10318           0
10319           0
Name: freight_cost_usd, dtype: object

### Invoice Separately

In [175]:
# convert 344 'Invoiced Separately' elements to nan
data['freight_cost_usd'] = data['freight_cost_usd'].replace('Invoiced Separately', np.nan)
data['freight_cost_usd'].isnull().sum() # check nans to confirm

344

In [176]:
# impute missing values using median for continous data 
freight_median = data['freight_cost_usd'].median()
freight_median

5046.29

In [177]:
data['freight_cost_usd'] = data['freight_cost_usd'].fillna(freight_median)

data.loc[data['freight_cost_usd'] == 5046.29]

,id,project_code,pq_num,po___so_num,asn_dn_num,country,managed_by,fulfill_via,vendor_inco_term,shipment_mode,...,unit_of_measure_per_pack,line_item_quantity,line_item_value,pack_price,unit_price,manufacturing_site,first_line_designation,weight_kilograms,freight_cost_usd,line_item_insurance_usd
12,62,102-NG-T01,Pre-PQ Process,SCMS-230,ASN-144,Nigeria,PMO - US,Direct Drop,EXW,Air,...,100,10,850.00,85.00,0.85,"EY Laboratories, USA",True,1303.0,5046.29,NaN
36,305,123-NG-T01,Pre-PQ Process,SCMS-15550,ASN-1269,Nigeria,PMO - US,Direct Drop,EXW,Air,...,100,650,6.50,0.01,0.00,Inverness Japan,True,1303.0,5046.29,0.01
38,361,125-HT-T01,Pre-PQ Process,SCMS-14080,ASN-1318,Haiti,PMO - US,Direct Drop,EXW,Air,...,100,2,278.00,139.00,1.39,"Trinity Biotech, Plc",True,1303.0,5046.29,0.44
41,398,102-GY-T01,Pre-PQ Process,SCMS-18480,ASN-1502,Guyana,PMO - US,Direct Drop,EXW,Air,...,100,240,2.40,0.01,0.00,Inverness Japan,True,1303.0,5046.29,0.00
53,465,102-NG-T01,Pre-PQ Process,SCMS-21540,ASN-2011,Nigeria,PMO - US,Direct Drop,EXW,Air,...,100,5498,54.98,0.01,0.00,Inverness Japan,True,1303.0,5046.29,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10167,86584,111-MZ-T30,FPQ-11033,SO-43774,DN-3230,Mozambique,PMO - US,From RDC,N/A - From RDC,Truck,...,30,50000,526000.00,10.52,0.35,Hetero Unit III Hyderabad IN,True,1303.0,5046.29,650.66
10168,86585,111-MZ-T30,FPQ-11033,SO-43774,DN-3230,Mozambique,PMO - US,From RDC,N/A - From RDC,Truck,...,30,1000,10520.00,10.52,0.35,Mylan (formerly Matrix) Nashik,False,1303.0,5046.29,13.01
10172,86589,102-BI-T30,FPQ-12065,SO-45151,DN-3260,Burundi,PMO - US,From RDC,N/A - From RDC,Air,...,30,48,225.60,4.70,0.16,Hetero Unit III Hyderabad IN,False,1303.0,5046.29,0.28
10178,86596,102-BI-T30,FPQ-12602,SO-46021,DN-3283,Burundi,PMO - US,From RDC,N/A - From RDC,Air,...,60,63,142.38,2.26,0.04,Hetero Unit III Hyderabad IN,True,1303.0,5046.29,0.18


In [178]:
# convert weight_kilograms & freight_cost_usd cols from object to float dtype
data[['weight_kilograms', 'freight_cost_usd']] = data[['weight_kilograms', 'freight_cost_usd']].astype(float)
data[['weight_kilograms', 'freight_cost_usd']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10320 entries, 0 to 10319
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   weight_kilograms  10320 non-null  float64
 1   freight_cost_usd  10320 non-null  float64
dtypes: float64(2)
memory usage: 161.4 KB


In [179]:
data[['freight_cost_usd', 'weight_kilograms']].tail()

,freight_cost_usd,weight_kilograms
10315,46111.55,25880.0
10316,14734.92,4426.0
10317,0.00,1303.0
10318,0.00,1392.0
10319,0.00,1303.0


## item_description

### Data Reduction

In [180]:
# sort alphabetically to view
# item_description = 184 unique items 
itemd = data.sort_values('item_description')
itemd['item_description'].unique()

array(['#102198**Didanosine 200mg [Videx], tablets, 60 Tabs',
       '#108853** HIV, Genie II HIV-1/HIV-2 Kit, 40 Tests',
       '#109435**HIV, OraQuick Controls Kit, 3 Vials',
       '#109436**HIV 1, Uni-Gold Recombigen HIV Control, Vial 2 x 0.5 ml',
       '#109440**HIV 1/2, INSTI HIV Antibody Individual POC, 1 Test',
       'Abacavir 20mg/ml [Ziagen], oral solution, Bottle, 240 ml',
       'Abacavir 20mg/ml, [DON] oral solution, Bottle, 240 ml',
       'Abacavir 20mg/ml, oral solution w/syringe, Bottle, 240 ml',
       'Abacavir 20mg/ml, oral solution, Bottle, 240 ml',
       'Abacavir 300mg [Ziagen], tablets, Blister 6 x 10 Tabs',
       'Abacavir 300mg, tablets, 60 Tabs',
       'Abacavir 60mg, dispersible tablets, 60 Tabs',
       'Abacavir/Lamivudine 60/30mg, dispersible tablets, 60 Tabs',
       'Abacavir/Lamivudine 60/30mg, tablets, 30 Tabs',
       'Abacavir/Lamivudine 60/30mg, tablets, 60 Tabs',
       'Abacavir/Lamivudine 600/300mg, scored tablets, 30 Tabs',
       'Abacavi

'item_description' & 'molecule_test_type' columns are almost identical. The item description is more detailed containing information about the dosage and unit of measure (per pack). These fields are already present as other columns in the dataset: 'molecule_test_type', 'dosage', 'dosage_form', & 'unit of measure (per pack)' That's why there are missing fields because some of the item descriptions do not contain the dosage and only the dosage form.

item_description = molecule_test_type, brand, dosage, dosage_form, unit_of_measure_per_pack

This, this col can be dropped

In [187]:
# Example 
data['item_description'][0]

'HIV, Reveal G3 Rapid HIV-1 Antibody Test, 30 Tests'

In [188]:
data.iloc[[0], [18, 19, 20, 21, 22, 23]]

,item_description,molecule_test_type,brand,dosage,dosage_form,unit_of_measure_per_pack
0,"HIV, Reveal G3 Rapid HIV-1 Antibody Test, 30 T...","HIV, Reveal G3 Rapid HIV-1 Antibody Test",Reveal,NaN,Test kit,30


In [189]:
# drop item_description column 
data.drop("item_description",axis=1,inplace=True)
data.head

<bound method NDFrame.head of           id project_code          pq_num po___so_num asn_dn_num  \
0          1   100-CI-T01  Pre-PQ Process      SCMS-4      ASN-8   
1          3   108-VN-T01  Pre-PQ Process     SCMS-13     ASN-85   
2          4   100-CI-T01  Pre-PQ Process     SCMS-20     ASN-14   
3         15   108-VN-T01  Pre-PQ Process     SCMS-78     ASN-50   
4         16   108-VN-T01  Pre-PQ Process     SCMS-81     ASN-55   
...      ...          ...             ...         ...        ...   
10315  86818   103-ZW-T30       FPQ-15197    SO-50020    DN-4307   
10316  86819   104-CI-T30       FPQ-15259    SO-50102    DN-4313   
10317  86821   110-ZM-T30       FPQ-14784    SO-49600    DN-4316   
10318  86822   200-ZW-T30       FPQ-16523    SO-51680    DN-4334   
10319  86823   103-ZW-T30       FPQ-15197    SO-50022    DN-4336   

             country managed_by  fulfill_via vendor_inco_term shipment_mode  \
0      Côte d'Ivoire   PMO - US  Direct Drop              EXW           Ai

Now there are 10320 rows, 32 columns after dropping 'item_description'

## molecule_test_type

In [74]:
data['molecule_test_type'].value_counts()

molecule_test_type
Efavirenz                           1123
Nevirapine                           877
Lamivudine/Nevirapine/Zidovudine     707
Lamivudine/Zidovudine                689
Lopinavir/Ritonavir                  633
                                    ... 
Mefloquine (as hydrochloride)          1
Quinine (as sulfate)                   1
Primaquine base (as diphosphate)       1
Lamivudine/Zidovudine+Abacavir         1
Nelfinavir                             1
Name: count, Length: 86, dtype: int64

In [75]:
# sort alphabetically to view
mtt = data.sort_values('molecule_test_type')
mtt['molecule_test_type'].unique()

array(['Abacavir', 'Abacavir/Lamivudine',
       'Abacavir/Lamivudine/Zidovudine', 'Artemether/Lumefantrine',
       'Artesunate+Amodiaquine', 'Atazanavir', 'Atazanavir/Ritonavir',
       'Chase Buffer, Determine, 100 Tests, 2.5ml x 1 Vial',
       'Chloroquine base (as phosphate)', 'Darunavir', 'Didanosine',
       'Didanosine EC', 'Efavirenz',
       'Efavirenz/Emtricitabine/Tenofovir Disoproxil Fumarate',
       'Efavirenz/Lamivudine/Tenofovir Disoproxil Fumarate',
       'Emtricitabine/Tenofovir Disoproxil Fumarate', 'Etravirine',
       'HIV 1, Uni-Gold Recombigen HIV Control, Vial 2 x 0.5 ml',
       'HIV 1/2, Bioline 3.0 Diluent, 25 Tests',
       'HIV 1/2, Bioline 3.0 Kit',
       'HIV 1/2, Bioline 3.0 Kit (no accessories)',
       'HIV 1/2, Bioline 3.0 Kit, Lancets, Capillary pipets, Alcohol swabs included',
       'HIV 1/2, Bundi Rapid HIV Kit', 'HIV 1/2, Capillus HIV Kit',
       'HIV 1/2, Clearview Complete Kit',
       'HIV 1/2, Colloidal Gold, Diagnostic Kit Set (includes

### Feature Engineering / Create Features

In [1]:
data['active_ingredients'].nunique()

NameError: name 'data' is not defined

In [76]:
data['active_ingredients'] = data['molecule_test_type'].str.split(',').str[0]
data['active_ingredients']

0                                                      HIV
1                                               Nevirapine
2                                                  HIV 1/2
3                                               Lamivudine
4                                                Stavudine
                               ...                        
10315                     Lamivudine/Nevirapine/Zidovudine
10316                                Lamivudine/Zidovudine
10317    Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...
10318                                Lamivudine/Zidovudine
10319                                Lamivudine/Zidovudine
Name: active_ingredients, Length: 10320, dtype: object

In [77]:
data['active_ingredients'].value_counts()

active_ingredients
HIV 1/2                                                  1519
Efavirenz                                                1123
Nevirapine                                                877
Lamivudine/Nevirapine/Zidovudine                          707
Lamivudine/Zidovudine                                     689
Lopinavir/Ritonavir                                       633
Lamivudine                                                592
Zidovudine                                                529
Abacavir                                                  453
Tenofovir Disoproxil Fumarate                             320
Lamivudine/Tenofovir Disoproxil Fumarate                  301
Efavirenz/Lamivudine/Tenofovir Disoproxil Fumarate        286
Lamivudine/Nevirapine/Stavudine                           286
Stavudine                                                 284
Didanosine                                                263
Emtricitabine/Tenofovir Disoproxil Fumarate        

In [78]:
data['active_ingredients'].isnull().sum()

0

In [79]:
data['molecule_test_type'] = data.apply(lambda row: row['molecule_test_type'].replace(row['brand'], '').strip(), axis=1)
data['molecule_test_type']

0                       HIV,  G3 Rapid HIV-1 Antibody Test
1                                               Nevirapine
2                               HIV 1/2,  Complete HIV Kit
3                                               Lamivudine
4                                                Stavudine
                               ...                        
10315                     Lamivudine/Nevirapine/Zidovudine
10316                                Lamivudine/Zidovudine
10317    Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...
10318                                Lamivudine/Zidovudine
10319                                Lamivudine/Zidovudine
Name: molecule_test_type, Length: 10320, dtype: object

In [80]:
# remove extra whitespace after remove brand from molecule_test_type
# HIV 1/2,  Complete HIV Kit
data['molecule_test_type'] = data['molecule_test_type'].str.replace(r'\s+', ' ', regex = True).str.strip()
data['molecule_test_type']

0                        HIV, G3 Rapid HIV-1 Antibody Test
1                                               Nevirapine
2                                HIV 1/2, Complete HIV Kit
3                                               Lamivudine
4                                                Stavudine
                               ...                        
10315                     Lamivudine/Nevirapine/Zidovudine
10316                                Lamivudine/Zidovudine
10317    Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...
10318                                Lamivudine/Zidovudine
10319                                Lamivudine/Zidovudine
Name: molecule_test_type, Length: 10320, dtype: object

In [81]:
# extract second part of the element after the ','
# if there is no comma or just one element after ',' then results in Nan for that row under 'description_type' col
data['description_type'] = data['molecule_test_type'].apply(lambda x: x.split(',')[1].strip() if (',' in x) and len(x.split(',')) > 1 else np.nan)
data['description_type']

0        G3 Rapid HIV-1 Antibody Test
1                                 NaN
2                    Complete HIV Kit
3                                 NaN
4                                 NaN
                     ...             
10315                             NaN
10316                             NaN
10317                             NaN
10318                             NaN
10319                             NaN
Name: description_type, Length: 10320, dtype: object

In [82]:
data['description_type'].isnull().sum()

8591

In [83]:
#sample = data['molecule_test_type'].apply(lambda x: x.split(',')[1].strip() if ',' in x else np.nan)
#sample.loc[data['freight_cost_usd'] == 5046.29]

# explore cols for observations
desc_na = data[data['description_type'].isna()]
desc_na[['molecule_test_type', 'active_ingredients', 'description_type']]

,molecule_test_type,active_ingredients,description_type
1,Nevirapine,Nevirapine,NaN
3,Lamivudine,Lamivudine,NaN
4,Stavudine,Stavudine,NaN
5,Zidovudine,Zidovudine,NaN
6,Efavirenz,Efavirenz,NaN
...,...,...,...
10315,Lamivudine/Nevirapine/Zidovudine,Lamivudine/Nevirapine/Zidovudine,NaN
10316,Lamivudine/Zidovudine,Lamivudine/Zidovudine,NaN
10317,Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...,Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...,NaN
10318,Lamivudine/Zidovudine,Lamivudine/Zidovudine,NaN


This shows when molecule_test_type is the same as active_ingredients there is no description provided because it is an active drug. Due to the nature of the dataset there is no additional information provided for these drugs such as Nevirapine, Lamivudine, etc.

Instead, convert NaN to 'None' of object dtype

In [84]:
# Convert NaN to None in description_type col because it's object dtype
data['description_type'] = data['description_type'].fillna(pd.NA).replace({pd.NA: None})
data[['molecule_test_type', 'active_ingredients', 'description_type']].head()

,molecule_test_type,active_ingredients,description_type
0,"HIV, G3 Rapid HIV-1 Antibody Test",HIV,G3 Rapid HIV-1 Antibody Test
1,Nevirapine,Nevirapine,None
2,"HIV 1/2, Complete HIV Kit",HIV 1/2,Complete HIV Kit
3,Lamivudine,Lamivudine,None
4,Stavudine,Stavudine,None


In [85]:
data['description_type'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10320 entries, 0 to 10319
Series name: description_type
Non-Null Count  Dtype 
--------------  ----- 
1729 non-null   object
dtypes: object(1)
memory usage: 80.8+ KB


In [86]:
data[['molecule_test_type', 'active_ingredients', 'description_type', 'brand', 'dosage', 'dosage_form', 'unit_of_measure_per_pack']].head()

,molecule_test_type,active_ingredients,description_type,brand,dosage,dosage_form,unit_of_measure_per_pack
0,"HIV, G3 Rapid HIV-1 Antibody Test",HIV,G3 Rapid HIV-1 Antibody Test,Reveal,NaN,Test kit,30
1,Nevirapine,Nevirapine,None,Generic,10mg/ml,Oral suspension,240
2,"HIV 1/2, Complete HIV Kit",HIV 1/2,Complete HIV Kit,Determine,NaN,Test kit,100
3,Lamivudine,Lamivudine,None,Generic,150mg,Tablet,60
4,Stavudine,Stavudine,None,Generic,30mg,Capsule,60


In [87]:
mtt_list = data['molecule_test_type'].tolist()
mtt_list

['HIV, G3 Rapid HIV-1 Antibody Test',
 'Nevirapine',
 'HIV 1/2, Complete HIV Kit',
 'Lamivudine',
 'Stavudine',
 'Zidovudine',
 'Efavirenz',
 'Nevirapine',
 'Stavudine',
 'Lopinavir/Ritonavir',
 'Lopinavir/Ritonavir',
 'HIV 1/2, HIV Kit',
 'HIV 1/2, InstantChek HIV 1+2 Kit',
 'Lamivudine/Zidovudine',
 'HIV 1/2, Complete HIV Kit',
 'Didanosine',
 'HIV 1/2, Complete HIV Kit',
 'HIV 1/2/O, HIV 1-2.0, v.3.0 Cards Kit',
 'Zidovudine',
 'Didanosine',
 'HIV 1/2, Complete HIV Kit',
 'HIV 1/2, HIV, Kit',
 'Didanosine',
 'Nevirapine',
 'Lopinavir/Ritonavir',
 'HIV 1/2, HIV Kit, without Lancets',
 'HIV 1/2, Advance HIV Rapid Antibody Kit',
 'HIV 1/2, 3.0 Kit, Lancets, Capillary pipets, Alcohol swabs included',
 'HIV 1/2, 3.0 Kit, Lancets, Capillary pipets, Alcohol swabs included',
 'HIV 1/2, 3.0 Kit, Lancets, Capillary pipets, Alcohol swabs included',
 'Stavudine',
 'HIV 1/2, HIV, Kit',
 'Zidovudine',
 'Tenofovir Disoproxil Fumarate',
 'Zidovudine',
 'HIV, Lancet, Safety, for HIV Test kits, 100 P

In [88]:
act_ing_list = data['active_ingredients'].tolist()
act_ing_list

['HIV',
 'Nevirapine',
 'HIV 1/2',
 'Lamivudine',
 'Stavudine',
 'Zidovudine',
 'Efavirenz',
 'Nevirapine',
 'Stavudine',
 'Lopinavir/Ritonavir',
 'Lopinavir/Ritonavir',
 'HIV 1/2',
 'HIV 1/2',
 'Lamivudine/Zidovudine',
 'HIV 1/2',
 'Didanosine',
 'HIV 1/2',
 'HIV 1/2/O',
 'Zidovudine',
 'Didanosine',
 'HIV 1/2',
 'HIV 1/2',
 'Didanosine',
 'Nevirapine',
 'Lopinavir/Ritonavir',
 'HIV 1/2',
 'HIV 1/2',
 'HIV 1/2',
 'HIV 1/2',
 'HIV 1/2',
 'Stavudine',
 'HIV 1/2',
 'Zidovudine',
 'Tenofovir Disoproxil Fumarate',
 'Zidovudine',
 'HIV',
 'HIV',
 'Stavudine',
 'HIV 1/2',
 'HIV',
 'HIV 1/2',
 'HIV',
 'HIV',
 'HIV 1/2',
 'HIV 1/2',
 'HIV 1/2',
 'HIV 1/2',
 'Efavirenz',
 'Saquinavir',
 'Stavudine',
 'Didanosine',
 'Didanosine',
 'HIV 1/2',
 'HIV',
 'HIV 1/2',
 'Tenofovir Disoproxil Fumarate',
 'Stavudine',
 'Didanosine EC',
 'Stavudine',
 'Didanosine EC',
 'HIV 1/2',
 'HIV 1/2',
 'Abacavir',
 'Lamivudine',
 'Artemether/Lumefantrine',
 'Stavudine',
 'Nevirapine',
 'Efavirenz',
 'Malaria Paramax

In [89]:
desc_list = data['description_type'].tolist()
desc_list

['G3 Rapid HIV-1 Antibody Test',
 None,
 'Complete HIV Kit',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'HIV Kit',
 'InstantChek HIV 1+2 Kit',
 None,
 'Complete HIV Kit',
 None,
 'Complete HIV Kit',
 'HIV 1-2.0',
 None,
 None,
 'Complete HIV Kit',
 'HIV',
 None,
 None,
 None,
 'HIV Kit',
 'Advance HIV Rapid Antibody Kit',
 '3.0 Kit',
 '3.0 Kit',
 '3.0 Kit',
 None,
 'HIV',
 None,
 None,
 None,
 'Lancet',
 'Lancet',
 None,
 'HIV Kit',
 'II HIV-1/HIV-2 Kit',
 '3.0 Kit',
 'Lancet',
 'Lancet',
 'HIV Kit',
 'HIV Kit',
 'HIV',
 'HIV Kit',
 None,
 None,
 None,
 None,
 None,
 'HIV Kit',
 'Lancet',
 'HIV Kit',
 None,
 None,
 None,
 None,
 None,
 'Advance HIV Rapid Antibody Kit',
 'HIV Kit',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'HIV Kit',
 'HIV Kit',
 'II HIV-1/HIV-2 Kit',
 None,
 None,
 'HIV Kit',
 'Recombigen HIV Control',
 '3.0 Kit',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'HIV Kit',
 None,
 None,
 None,
 None,
 None,
 None,
 'HIV 1-2.0',
 Non

In [90]:
Counter(desc_list)

Counter({None: 8591,
         'Complete HIV Kit': 577,
         'HIV Kit': 541,
         '': 110,
         'HIV': 98,
         '3.0 Kit': 88,
         'Lancet': 65,
         'Advance HIV Rapid Antibody Kit': 47,
         'Capillary Tubes': 34,
         'II HIV-1/HIV-2 Kit': 23,
         'Complete Kit': 19,
         '3.0 Kit (no accessories)': 19,
         'HIV 1-2.0': 15,
         'Dipstick Assay Kit': 15,
         'Controls Kit': 11,
         'Kit without Lancets': 9,
         'III Kit': 7,
         'HIV-1/HIV-2 Rapid Test Kit': 5,
         'Recombigen HIV Control': 4,
         'INSTI HIV Antibody Individual POC': 4,
         'New Blot 2 HIV Kit': 4,
         '3.0 Diluent': 4,
         'G3 Rapid HIV-1 Antibody Test': 3,
         'Gold Kit': 3,
         'HEXAGON Rapid HIV Kit': 3,
         'InstantChek HIV 1+2 Kit': 2,
         'II Kit': 2,
         'Rapid HIV Kit': 2,
         'TriSpot Ag - Ab Kit': 2,
         'HIV 1/2 3.0 Kit': 2,
         'Rapid Antibody Kit': 2,
         'Buffer O

In [91]:
data.isnull().sum()

id                                 0
project_code                       0
pq_num                             0
po___so_num                        0
asn_dn_num                         0
country                            0
managed_by                         0
fulfill_via                        0
vendor_inco_term                   0
shipment_mode                    360
pq_first_sent_to_client_date       0
po_sent_to_vendor_date             0
scheduled_delivery_date            0
delivered_to_client_date           0
delivery_recorded_date             0
product_group                      0
sub_classification                 0
vendor                             0
molecule_test_type                 0
brand                              0
dosage                          1734
dosage_form                        0
unit_of_measure_per_pack           0
line_item_quantity                 0
line_item_value                    0
pack_price                         0
unit_price                         0
m

In [92]:
data

,id,project_code,pq_num,po___so_num,asn_dn_num,country,managed_by,fulfill_via,vendor_inco_term,shipment_mode,...,line_item_value,pack_price,unit_price,manufacturing_site,first_line_designation,weight_kilograms,freight_cost_usd,line_item_insurance_usd,active_ingredients,description_type
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,551.00,29.00,0.97,Ranbaxy Fine Chemicals LTD,True,13.0,780.34,NaN,HIV,G3 Rapid HIV-1 Antibody Test
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,6200.00,6.20,0.03,"Aurobindo Unit III, India",True,358.0,4521.50,NaN,Nevirapine,None
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,40000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,True,171.0,1653.78,NaN,HIV 1/2,Complete HIV Kit
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,127360.80,3.99,0.07,"Ranbaxy, Paonta Shahib, India",True,1855.0,16007.06,NaN,Lamivudine,None
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,121600.00,3.20,0.05,"Aurobindo Unit III, India",True,7590.0,45450.08,NaN,Stavudine,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10315,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",False,25880.0,46111.55,705.79,Lamivudine/Nevirapine/Zidovudine,None
10316,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,...,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,False,4426.0,14734.92,161.71,Lamivudine/Zidovudine,None
10317,86821,110-ZM-T30,FPQ-14784,SO-49600,DN-4316,Zambia,PMO - US,From RDC,N/A - From RDC,Truck,...,5140114.74,9.99,0.33,Cipla Ltd A-42 MIDC Mahar. IN,False,1303.0,0.00,5284.04,Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...,None
10318,86822,200-ZW-T30,FPQ-16523,SO-51680,DN-4334,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,113871.80,6.52,0.11,Mylan (formerly Matrix) Nashik,True,1392.0,0.00,134.03,Lamivudine/Zidovudine,None


In [93]:
data.columns

Index(['id', 'project_code', 'pq_num', 'po___so_num', 'asn_dn_num', 'country',
       'managed_by', 'fulfill_via', 'vendor_inco_term', 'shipment_mode',
       'pq_first_sent_to_client_date', 'po_sent_to_vendor_date',
       'scheduled_delivery_date', 'delivered_to_client_date',
       'delivery_recorded_date', 'product_group', 'sub_classification',
       'vendor', 'molecule_test_type', 'brand', 'dosage', 'dosage_form',
       'unit_of_measure_per_pack', 'line_item_quantity', 'line_item_value',
       'pack_price', 'unit_price', 'manufacturing_site',
       'first_line_designation', 'weight_kilograms', 'freight_cost_usd',
       'line_item_insurance_usd', 'active_ingredients', 'description_type'],
      dtype='object')

In [94]:
# 0 - 33
data.columns[0]

'id'

## manufacturing_site

88 unique manufacturing sites that partner with USAID and ship commodities and treatments for HIV/AIDS, Malaria

### Data Cleaning / Wrangling

In [190]:
print(data.manufacturing_site.nunique())

88


In [191]:
# sort alphabetically to view
manufacturing = data.sort_values('manufacturing_site')
manufacturing['manufacturing_site'].unique()

array(['ABBSP', 'ABBVIE (Abbott) France',
       'ABBVIE (Abbott) Japan Co. Ltd.', 'ABBVIE (Abbott) Logis. UK',
       "ABBVIE (Abbott) St. P'burg USA", 'ABBVIE GmbH & Co.KG Wiesbaden',
       'ABBVIE Labs North Chicago US', 'ABBVIE Ludwigshafen Germany',
       'Access BIO, L.C.', 'Alere Medical Co., Ltd.',
       'Aspen-OSD, Port Elizabeth, SA', 'Aurobindo Unit III, India',
       'Aurobindo Unit VII, IN', 'BI, Ingelheim, Germany',
       'BMS Evansville, US', 'BMS Meymac, France',
       'BUNDI INTERNATIONAL DIAGNOSTICS LTD', 'Bio-Rad Laboratories',
       'Boehringer Ing., Koropi, GR', 'Boehringer Ingelheim Roxane US',
       'Bristol-Myers Squibb Anagni IT', 'Chembio Diagnostics Sys. Inc.',
       'Cipla Ltd A-42 MIDC Mahar. IN', 'Cipla, Goa, India',
       'Cipla, Kurkumbh, India', 'Cipla, Patalganga, India',
       'EY Laboratories, USA',
       'Emcure Plot No.P-2, I.T-B.T. Park, Phase II, MIDC, Hinjwadi, Pune, India',
       'GSK Aranda', 'GSK Barnard Castle UK',
       'GSK C

There is only one ABBSP in the dataset. After some exploration we see ABBSP is part of Abbvie (Abbott) Pharmaceuticals based on the vendor 'ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV)'. After some research to gain domain knowledge, we could not find any abbreviations related to pharmaceuticals, supply chains, or if it was related to abbvie or abbott. So we can assume ABBSP is a typo. 

Since we are cleaning the data we will place this with 'Abbvie' and leave the location as None since it is not known. 

In [192]:
data[['vendor','manufacturing_site']].loc[data['manufacturing_site'] == 'ABBSP']

,vendor,manufacturing_site
140,ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV),ABBSP


In [193]:
data[['vendor','manufacturing_site']].loc[data['vendor'] == 'ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV)']

,vendor,manufacturing_site
9,ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV),ABBVIE (Abbott) St. P'burg USA
10,ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV),ABBVIE Ludwigshafen Germany
24,ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV),ABBVIE Ludwigshafen Germany
91,ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV),ABBVIE (Abbott) Logis. UK
92,ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV),ABBVIE Ludwigshafen Germany
...,...,...
6500,ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV),ABBVIE (Abbott) Logis. UK
6504,ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV),ABBVIE Ludwigshafen Germany
6506,ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV),ABBVIE (Abbott) Logis. UK
6514,ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV),ABBVIE (Abbott) Logis. UK


In [194]:
# print rows with  'Ranbaxy per Shasun Pharma' or 'Ranbaxy per Shasun Pharma Ltd'
data[['vendor','manufacturing_site']].loc[data['manufacturing_site'] == 'Ranbaxy per Shasun Pharma']

,vendor,manufacturing_site
6954,SCMS from RDC,Ranbaxy per Shasun Pharma
7008,SCMS from RDC,Ranbaxy per Shasun Pharma
8787,SCMS from RDC,Ranbaxy per Shasun Pharma
8830,SCMS from RDC,Ranbaxy per Shasun Pharma
9306,SCMS from RDC,Ranbaxy per Shasun Pharma
9316,SCMS from RDC,Ranbaxy per Shasun Pharma
10240,SCMS from RDC,Ranbaxy per Shasun Pharma
10262,SCMS from RDC,Ranbaxy per Shasun Pharma
10288,SCMS from RDC,Ranbaxy per Shasun Pharma


In [195]:
data[['vendor','manufacturing_site']].loc[data['manufacturing_site'] == 'Ranbaxy per Shasun Pharma Ltd']

,vendor,manufacturing_site
6193,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,Ranbaxy per Shasun Pharma Ltd
8315,SCMS from RDC,Ranbaxy per Shasun Pharma Ltd


In [196]:
data[['vendor','manufacturing_site']].loc[data['manufacturing_site'] == 'Ranbaxy Fine Chemicals LTD']

,vendor,manufacturing_site
0,RANBAXY Fine Chemicals LTD.,Ranbaxy Fine Chemicals LTD
524,RANBAXY Fine Chemicals LTD.,Ranbaxy Fine Chemicals LTD


In [197]:
data[['vendor','manufacturing_site']].loc[data['manufacturing_site'] == 'Ranbaxy, Paonta Shahib, India'].head(10)

,vendor,manufacturing_site
3,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,"Ranbaxy, Paonta Shahib, India"
34,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,"Ranbaxy, Paonta Shahib, India"
530,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,"Ranbaxy, Paonta Shahib, India"
532,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,"Ranbaxy, Paonta Shahib, India"
660,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,"Ranbaxy, Paonta Shahib, India"
674,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,"Ranbaxy, Paonta Shahib, India"
785,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,"Ranbaxy, Paonta Shahib, India"
920,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,"Ranbaxy, Paonta Shahib, India"
1045,SCMS from RDC,"Ranbaxy, Paonta Shahib, India"
1049,SCMS from RDC,"Ranbaxy, Paonta Shahib, India"


In [198]:
manufacturing['manufacturing_site'].unique()

array(['ABBSP', 'ABBVIE (Abbott) France',
       'ABBVIE (Abbott) Japan Co. Ltd.', 'ABBVIE (Abbott) Logis. UK',
       "ABBVIE (Abbott) St. P'burg USA", 'ABBVIE GmbH & Co.KG Wiesbaden',
       'ABBVIE Labs North Chicago US', 'ABBVIE Ludwigshafen Germany',
       'Access BIO, L.C.', 'Alere Medical Co., Ltd.',
       'Aspen-OSD, Port Elizabeth, SA', 'Aurobindo Unit III, India',
       'Aurobindo Unit VII, IN', 'BI, Ingelheim, Germany',
       'BMS Evansville, US', 'BMS Meymac, France',
       'BUNDI INTERNATIONAL DIAGNOSTICS LTD', 'Bio-Rad Laboratories',
       'Boehringer Ing., Koropi, GR', 'Boehringer Ingelheim Roxane US',
       'Bristol-Myers Squibb Anagni IT', 'Chembio Diagnostics Sys. Inc.',
       'Cipla Ltd A-42 MIDC Mahar. IN', 'Cipla, Goa, India',
       'Cipla, Kurkumbh, India', 'Cipla, Patalganga, India',
       'EY Laboratories, USA',
       'Emcure Plot No.P-2, I.T-B.T. Park, Phase II, MIDC, Hinjwadi, Pune, India',
       'GSK Aranda', 'GSK Barnard Castle UK',
       'GSK C

ISO 2 letter Country Codes from https://www.iban.com/country-codes

All Orasure Technologies are located in the US according to their website. Trinity Biotech is in Bray, Co Wicklow, Ireland. Ireland's country code is 'IE'. 

In [199]:
data['manufacturing_site'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10320 entries, 0 to 10319
Series name: manufacturing_site
Non-Null Count  Dtype 
--------------  ----- 
10320 non-null  object
dtypes: object(1)
memory usage: 80.8+ KB


In [200]:
# manufacturing_locations with ISO Country Codes (US = USA = United States, DE = Germany)
data['manufacturing_location'] = data['manufacturing_site'].replace({'ABBSP':'None','ABBVIE (Abbott) France':'FR', 'ABBVIE (Abbott) Japan Co. Ltd.':'JP', 
     'ABBVIE (Abbott) Logis. UK':'GB', "'ABBVIE (Abbott) St. P'burg USA'":'St. Petersburg, US',
     'ABBVIE GmbH & Co.KG Wiesbaden':'Wiesbaden, DE', 'ABBVIE Labs North Chicago US':'North Chicago, US',
     'ABBVIE Ludwigshafen Germany':'Ludwigshafen, DE', 'Access BIO, L.C.':'New Jersey, US', 
     'Alere Medical Co., Ltd.':'Massachusetts, US', 'Aspen-OSD, Port Elizabeth, SA':'Port Elizabeth, SA',
     'Aurobindo Unit III, India':'Telangana, IN', 'Aurobindo Unit VII, IN':'Jedcherla, Hyderaba IN', 
     'BI, Ingelheim, Germany':'Ingelheim, DE', 'BMS Evansville, US':'Evansville, US', 'BMS Meymac, France':'Meymac, France',
     'BUNDI INTERNATIONAL DIAGNOSTICS LTD':'Bundi International Diagnostics', 'Bio-Rad Laboratories':'None', 'Boehringer Ing., Koropi, GR':'Koropi, GR',
     'Boehringer Ingelheim Roxane US':'Roxane, US', 'Bristol-Myers Squibb Anagni IT':'Anagni, IT', 'Chembio Diagnostics Sys. Inc.':'NY, US',
     'Cipla Ltd A-42 MIDC Mahar. IN':'Maharashtra, IN', 'Cipla, Goa, India':'Goa, IN', 'Cipla, Kurkumbh, India':'Kurkumbh, IN', 
     'Cipla, Patalganga, India':'Patalganga, IN', 'EY Laboratories, USA':'US',
     'Emcure Plot No.P-2, I.T-B.T. Park, Phase II, MIDC, Hinjwadi, Pune, India':'Hinjwadi, Pune, IN', 
     'GSK Aranda':'Aranda, ES', 'GSK Barnard Castle UK':'Barnard Castle, GB', 'GSK Cape Town Factory (South Africa)':'Cape Town, ZA', 
     'GSK Crawley':'Crawley, GB', 'GSK Mississauga (Canada)':'Mississauga, CA', 'GSK Ware (UK)':'Ware, Hertfordshire, GB', 
     'GSK, U1, Poznan, Poland':'Poznan, PL', 'Gilead(Nycomed) Oranienburg DE':'Oranienburg, DE', 'Gland Pharma Ltd Pally Factory':'IN', 
     'Gland Pharma, Hyderabad, IN':'Hyderabad, IN', 'Guilin OSD site, No 17, China':'Guangxi Zhuang, CN', 
     'Hetero Unit III Hyderabad IN':'Hyderabad, IN', 'Hetero, Jadcherla, unit 5, IN':'Telangana, IN', 
     'Human Diagnostic':'Magdeburg, DE', 'INVERNESS ANY':'None', 'INVERNESS ORGENICS LINE':'Massachusetts, US', 'Inverness Japan':'JP', 
     'Inverness USA':'US', 'Ipca Dadra/Nagar Haveli IN':'Dadra and Nagar Haveli, IN', 'Janssen Ortho LLC, Puerto Rico':'Gurabo, PR', 
     'Janssen-Cilag, Latina, IT':'Latina, IT', 'KHB Test Kit Facility, Shanghai China':'Shanghai, CN', 'MSD Elkton USA':'Elkton, US', 
     'MSD Manati':'PR, US', "'MSD Midrand, J'burg, SA'":'Midrand, SA', 'MSD Patheon, Canada':'Patheon, CA', 
     'MSD South Granville Australia':'Granville, AU', 'MSD, Haarlem, NL':'Haarlem, NL', 'Macleods Daman Plant INDIA':'Daman, IN',
     'MedMira Inc.':'None', 'Meditab (for Cipla) Daman IN':'Daman, IN', 'Medochemie Factory A, CY':'Limassol, CY', 
     'Medopharm Malur Factory, INDIA':'Malur, IN', 'Mepro Pharm Wadhwan Unit II':'Wadhwan, IN', 'Micro Labs Ltd. (Brown & Burk), India':'Bengaluru, IN', 
     'Micro Labs, Hosur, India':'Hosur, IN', 'Micro labs, Verna, Goa':'Goa, India', 'Mylan (formerly Matrix) Nashik':'Nashik, IN', 
     'Mylan,  H-12 & H-13, India':'IN', 'Not Applicable':'Not Applicable', 'Novartis Pharma AG, Switzerland':'CH', 
     'Novartis Pharma Suffern, USA':'US', 'OMEGA Diagnostics, UK':'Ely, GB', 'Orasure Technologies, Inc':'US', 
     'Orasure Technologies, Inc USA':'US', 'Pacific Biotech, Thailand':'TH', 'Premier Med. Corp Ltd. India':'IN', 
     'Premier Medical Corporation':'Daman, IN', 'Ranbaxy Fine Chemicals LTD':'Delhi, IN', 'Ranbaxy per Shasun Pharma':'IN',
     'Ranbaxy per Shasun Pharma Ltd':'IN', 'Ranbaxy, Paonta Shahib, India':'Paonta Sahib, IN', 'Remedica, Limassol, Cyprus':'Limassol, CY',
     'Roche Basel':'Basel, CH', 'Roche Madrid':'Madrid, ES', 'Standard Diagnostics, Korea':'KR',
     'Strides, Bangalore, India.':'Bangalore, IN', 'Trinity Biotech, Plc':'IE', 'Weifa A.S., Hausmanngt. 6, P.O. Box 9113 GrÃ¸nland, 0133, Oslo, Norway':'Oslo, NO',
     'bioLytical Laboratories':'CA'
    })

data[['manufacturing_site', 'manufacturing_location']]

,manufacturing_site,manufacturing_location
0,Ranbaxy Fine Chemicals LTD,"Delhi, IN"
1,"Aurobindo Unit III, India","Telangana, IN"
2,ABBVIE GmbH & Co.KG Wiesbaden,"Wiesbaden, DE"
3,"Ranbaxy, Paonta Shahib, India","Paonta Sahib, IN"
4,"Aurobindo Unit III, India","Telangana, IN"
...,...,...
10315,"Mylan, H-12 & H-13, India",IN
10316,Hetero Unit III Hyderabad IN,"Hyderabad, IN"
10317,Cipla Ltd A-42 MIDC Mahar. IN,"Maharashtra, IN"
10318,Mylan (formerly Matrix) Nashik,"Nashik, IN"


In [201]:
# extract manufacturers element only
data['manufacturers'] = data['manufacturing_site'].replace({'ABBSP':'Abbvie','ABBVIE (Abbott) France':'Abbvie', 'ABBVIE (Abbott) Japan Co. Ltd.':'Abbvie', 
     'ABBVIE (Abbott) Logis. UK':'Abbvie', "'ABBVIE (Abbott) St. P'burg USA'":'Abbvie',
     'ABBVIE GmbH & Co.KG Wiesbaden':'Abbvie', 'ABBVIE Labs North Chicago US':'Abbvie',
     'ABBVIE Ludwigshafen Germany':'Abbvie', 'Access BIO, L.C.':'Access BIO', 
     'Alere Medical Co., Ltd.':'Alere Medical', 'Aspen-OSD, Port Elizabeth, SA':'Aspen-OSD',
     'Aurobindo Unit III, India':'Aurobindo', 'Aurobindo Unit VII, IN':'Aurobindo', 
     'BI, Ingelheim, Germany':'Boehringer Ingelheim', 'BMS Evansville, US':'Bristol-Myers Squibb', 'BMS Meymac, France':'Bristol-Myers Squibb',
     'BUNDI INTERNATIONAL DIAGNOSTICS LTD':'Bundi International Diagnostics', 'Bio-Rad Laboratories':'Bio-Rad Laboratories', 'Boehringer Ing., Koropi, GR':'Boehringer Ingelheim',
     'Boehringer Ingelheim Roxane US':'Boehringer Ingelheim', 'Bristol-Myers Squibb Anagni IT':'Bristol-Myers Squibb', 'Chembio Diagnostics Sys. Inc.':'Chembio Diagnostics',
     'Cipla Ltd A-42 MIDC Mahar. IN':'Cipla', 'Cipla, Goa, India':'Cipla', 'Cipla, Kurkumbh, India':'Cipla', 
     'Cipla, Patalganga, India':'Cipla', 'EY Laboratories, USA':'EY Laboratories',
     'Emcure Plot No.P-2, I.T-B.T. Park, Phase II, MIDC, Hinjwadi, Pune, India':'Emcure Pharmaceuticals', 
     'GSK Aranda':'GSK', 'GSK Barnard Castle UK':'GSK', 'GSK Cape Town Factory (South Africa)':'GSK', 
     'GSK Crawley':'GSK', 'GSK Mississauga (Canada)':'GSK', 'GSK Ware (UK)':'GSK', 
     'GSK, U1, Poznan, Poland':'Poznan, PL', 'Gilead(Nycomed) Oranienburg DE':'Oranienburg, DE', 'Gland Pharma Ltd Pally Factory':'Gland Pharmaceuticals', 
     'Gland Pharma, Hyderabad, IN':'Gland Pharmaceuticals', 'Guilin OSD site, No 17, China':'Guilin OSD', 
     'Hetero Unit III Hyderabad IN':'Hetero', 'Hetero, Jadcherla, unit 5, IN':'Hetero', 
     'Human Diagnostic':'Human Diagnostic', 'INVERNESS ANY':'Iverness', 'INVERNESS ORGENICS LINE':'Iverness', 'Inverness Japan':'Iverness', 
     'Inverness USA':'Iverness', 'Ipca Dadra/Nagar Haveli IN':'Ipca', 'Janssen Ortho LLC, Puerto Rico':'Janssen Ortho', 
     'Janssen-Cilag, Latina, IT':'Janssen-Cilag', 'KHB Test Kit Facility, Shanghai China':'Shanghai Kehua Bio-Engineering', 'MSD Elkton USA':'MSD', 
     'MSD Manati':'MSD', "'MSD Midrand, J'burg, SA'":'MSD', 'MSD Patheon, Canada':'MSD', 
     'MSD South Granville Australia':'MSD', 'MSD, Haarlem, NL':'MSD', 'Macleods Daman Plant INDIA':'Macleods Pharmaceuticals',
     'MedMira Inc.':'MedMira', 'Meditab (for Cipla) Daman IN':'Cipla', 'Medochemie Factory A, CY':'Medochemie', 
     'Medopharm Malur Factory, INDIA':'Medopharm Pharmaceuticals', 'Mepro Pharm Wadhwan Unit II':'Mepro Pharmaceuticals', 'Micro Labs Ltd. (Brown & Burk), India':'Micro Labs', 
     'Micro Labs, Hosur, India':'Micro Labs', 'Micro labs, Verna, Goa':'Micro Labs', 'Mylan (formerly Matrix) Nashik':'Mylan', 
     'Mylan,  H-12 & H-13, India':'Mylan', 'Not Applicable':'Not Applicable', 'Novartis Pharma AG, Switzerland':'Novartis Pharmaceuticals', 
     'Novartis Pharma Suffern, USA':'Novartis Pharmaceuticals', 'OMEGA Diagnostics, UK':'Ely, GB', 'Orasure Technologies, Inc':'US', 
     'Orasure Technologies, Inc USA':'US', 'Pacific Biotech, Thailand':'TH', 'Premier Med. Corp Ltd. India':'IN', 
     'Premier Medical Corporation':'Daman, IN', 'Ranbaxy Fine Chemicals LTD':'Ranbaxy', 'Ranbaxy per Shasun Pharma':'Ranbaxy',
     'Ranbaxy per Shasun Pharma Ltd':'Ranbaxy', 'Ranbaxy, Paonta Shahib, India':'Ranbaxy', 'Remedica, Limassol, Cyprus':'Remedica',
     'Roche Basel':'Roche', 'Roche Madrid':'Roche', 'Standard Diagnostics, Korea':'Standard Diagnostics',
     'Strides, Bangalore, India.':'Strides', 'Trinity Biotech, Plc':'Trinity Biotech', 'Weifa A.S., Hausmanngt. 6, P.O. Box 9113 GrÃ¸nland, 0133, Oslo, Norway':'Weifa AS',
     'bioLytical Laboratories':'bioLytical Laboratories'
    })

data[['manufacturing_site', 'manufacturers']]

,manufacturing_site,manufacturers
0,Ranbaxy Fine Chemicals LTD,Ranbaxy
1,"Aurobindo Unit III, India",Aurobindo
2,ABBVIE GmbH & Co.KG Wiesbaden,Abbvie
3,"Ranbaxy, Paonta Shahib, India",Ranbaxy
4,"Aurobindo Unit III, India",Aurobindo
...,...,...
10315,"Mylan, H-12 & H-13, India",Mylan
10316,Hetero Unit III Hyderabad IN,Hetero
10317,Cipla Ltd A-42 MIDC Mahar. IN,Cipla
10318,Mylan (formerly Matrix) Nashik,Mylan


KHB Test Kit Facility = Shanghai Kehua Bio-Engineering according to https://www.prnewswire.com/news-releases/khb-becomes-first-chinese-company-to-obtain-eu-class-d-ivdr-certification-for-its-rapid-test-hiv-colloidal-gold-diagnostic-kit-301953311.html

MSD = Merck & Co Inc, we'll just leave as MSD for simplicity

According to https://www.outsourcing-pharma.com/Article/2010/08/25/Cipla-to-buy-Meditab-for-28m, Cipla bought Meditab and if we explore we see there is only one Meditab manufacturing_site, so we can just name this as Cipla

In [203]:
data[['manufacturing_site', 'manufacturers', 'manufacturing_location']].head()

,manufacturing_site,manufacturers,manufacturing_location
0,Ranbaxy Fine Chemicals LTD,Ranbaxy,"Delhi, IN"
1,"Aurobindo Unit III, India",Aurobindo,"Telangana, IN"
2,ABBVIE GmbH & Co.KG Wiesbaden,Abbvie,"Wiesbaden, DE"
3,"Ranbaxy, Paonta Shahib, India",Ranbaxy,"Paonta Sahib, IN"
4,"Aurobindo Unit III, India",Aurobindo,"Telangana, IN"


In [84]:
# We only see one element for Meditab (for Cipla)
data[['vendor','manufacturing_site']].loc[data['manufacturing_site'] == 'Meditab (for Cipla) Daman IN']

,vendor,manufacturing_site
606,IDA FOUNDATION,Meditab (for Cipla) Daman IN


In [87]:
# description_type has 8591 nulls, which is high because not all molecule_test_types have extra descriptions, especially active ingredients 
data.isnull().sum()
# description_type is just for observations and is not needed, can drop if needed or make further observations/EDA 

id                                 0
project_code                       0
pq_num                             0
po___so_num                        0
asn_dn_num                         0
country                            0
managed_by                         0
fulfill_via                        0
vendor_inco_term                   0
shipment_mode                    360
pq_first_sent_to_client_date       0
po_sent_to_vendor_date             0
scheduled_delivery_date            0
delivered_to_client_date           0
delivery_recorded_date             0
product_group                      0
sub_classification                 0
vendor                             0
molecule_test_type                 0
brand                              0
dosage                          1734
dosage_form                        0
unit_of_measure_per_pack           0
line_item_quantity                 0
line_item_value                    0
pack_price                         0
unit_price                         0
m

In [88]:
data

,id,project_code,pq_num,po___so_num,asn_dn_num,country,managed_by,fulfill_via,vendor_inco_term,shipment_mode,...,unit_price,manufacturing_site,first_line_designation,weight_kilograms,freight_cost_usd,line_item_insurance_usd,active_ingredients,description_type,manufacturing_location,manufacturers
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,0.97,Ranbaxy Fine Chemicals LTD,True,13.0,780.34,NaN,HIV,G3 Rapid HIV-1 Antibody Test,"Delhi, IN",Ranbaxy
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,0.03,"Aurobindo Unit III, India",True,358.0,4521.50,NaN,Nevirapine,None,"Telangana, IN",Aurobindo
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,0.80,ABBVIE GmbH & Co.KG Wiesbaden,True,171.0,1653.78,NaN,HIV 1/2,Complete HIV Kit,"Wiesbaden, DE",Abbvie
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,0.07,"Ranbaxy, Paonta Shahib, India",True,1855.0,16007.06,NaN,Lamivudine,None,"Paonta Sahib, IN",Ranbaxy
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,0.05,"Aurobindo Unit III, India",True,7590.0,45450.08,NaN,Stavudine,None,"Telangana, IN",Aurobindo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10315,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,0.06,"Mylan, H-12 & H-13, India",False,25880.0,46111.55,705.79,Lamivudine/Nevirapine/Zidovudine,None,IN,Mylan
10316,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,...,0.11,Hetero Unit III Hyderabad IN,False,4426.0,14734.92,161.71,Lamivudine/Zidovudine,None,"Hyderabad, IN",Hetero
10317,86821,110-ZM-T30,FPQ-14784,SO-49600,DN-4316,Zambia,PMO - US,From RDC,N/A - From RDC,Truck,...,0.33,Cipla Ltd A-42 MIDC Mahar. IN,False,1303.0,0.00,5284.04,Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...,None,"Maharashtra, IN",Cipla
10318,86822,200-ZW-T30,FPQ-16523,SO-51680,DN-4334,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,0.11,Mylan (formerly Matrix) Nashik,True,1392.0,0.00,134.03,Lamivudine/Zidovudine,None,"Nashik, IN",Mylan


molecule_test_type = active_ingredients + description_type + brand + dosage + dosage_form
* There are no dosage amounts for test kits so there will be blanks

manufacturing_site = manufacturers + manufacturing_location
* manufacturing_site will contain more info such as 'Ranbaxy Fine Chemicals LTD' instead of 'Ranbaxy' + 'Delhi, IN'
* Researched to gain domain knowledge to determine manufacturing_location if location was not provided in manufacturing_site col

In [95]:
data['po_sent_to_vendor_date'] = data['po_sent_to_vendor_date'].replace(['Date Not Captured', 'N/A - From RDC'],np.nan)
data['pq_first_sent_to_client_date'] = data['po_sent_to_vendor_date'].replace(['Date Not Captured', 'Pre-PQ Process'],np.nan)

In [96]:
date_cols = ['po_sent_to_vendor_date',
 'scheduled_delivery_date',
 'delivered_to_client_date',
 'delivery_recorded_date',
 'pq_first_sent_to_client_date'
 ]
display(data[date_cols])
for d in date_cols:
    print(d)
    data[d] = pd.to_datetime(data[d])

,po_sent_to_vendor_date,scheduled_delivery_date,delivered_to_client_date,delivery_recorded_date,pq_first_sent_to_client_date
0,NaN,2-Jun-06,2-Jun-06,2-Jun-06,NaN
1,NaN,14-Nov-06,14-Nov-06,14-Nov-06,NaN
2,NaN,27-Aug-06,27-Aug-06,27-Aug-06,NaN
3,NaN,1-Sep-06,1-Sep-06,1-Sep-06,NaN
4,NaN,11-Aug-06,11-Aug-06,11-Aug-06,NaN
...,...,...,...,...,...
10315,NaN,31-Jul-15,15-Jul-15,20-Jul-15,NaN
10316,NaN,31-Jul-15,6-Aug-15,7-Aug-15,NaN
10317,NaN,31-Aug-15,25-Aug-15,3-Sep-15,NaN
10318,NaN,9-Sep-15,4-Aug-15,11-Aug-15,NaN


po_sent_to_vendor_date
scheduled_delivery_date


C:\Users\Bertha\AppData\Local\Temp\ipykernel_6588\2910687795.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[d] = pd.to_datetime(data[d])


delivered_to_client_date


C:\Users\Bertha\AppData\Local\Temp\ipykernel_6588\2910687795.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[d] = pd.to_datetime(data[d])


delivery_recorded_date


C:\Users\Bertha\AppData\Local\Temp\ipykernel_6588\2910687795.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[d] = pd.to_datetime(data[d])


pq_first_sent_to_client_date


In [204]:
data[date_cols].head()

,po_sent_to_vendor_date,scheduled_delivery_date,delivered_to_client_date,delivery_recorded_date,pq_first_sent_to_client_date
0,Date Not Captured,2-Jun-06,2-Jun-06,2-Jun-06,Pre-PQ Process
1,Date Not Captured,14-Nov-06,14-Nov-06,14-Nov-06,Pre-PQ Process
2,Date Not Captured,27-Aug-06,27-Aug-06,27-Aug-06,Pre-PQ Process
3,Date Not Captured,1-Sep-06,1-Sep-06,1-Sep-06,Pre-PQ Process
4,Date Not Captured,11-Aug-06,11-Aug-06,11-Aug-06,Pre-PQ Process


In [97]:
data[date_cols].isnull().sum()

po_sent_to_vendor_date          5729
scheduled_delivery_date            0
delivered_to_client_date           0
delivery_recorded_date             0
pq_first_sent_to_client_date    5729
dtype: int64

In [98]:
# after converting to datetime data type there are now 5 cols with null values 
data.isnull().sum()

id                                 0
project_code                       0
pq_num                             0
po___so_num                        0
asn_dn_num                         0
country                            0
managed_by                         0
fulfill_via                        0
vendor_inco_term                   0
shipment_mode                    360
pq_first_sent_to_client_date    5729
po_sent_to_vendor_date          5729
scheduled_delivery_date            0
delivered_to_client_date           0
delivery_recorded_date             0
product_group                      0
sub_classification                 0
vendor                             0
molecule_test_type                 0
brand                              0
dosage                          1734
dosage_form                        0
unit_of_measure_per_pack           0
line_item_quantity                 0
line_item_value                    0
pack_price                         0
unit_price                         0
m

In [99]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10320 entries, 0 to 10319
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            10320 non-null  int64         
 1   project_code                  10320 non-null  object        
 2   pq_num                        10320 non-null  object        
 3   po___so_num                   10320 non-null  object        
 4   asn_dn_num                    10320 non-null  object        
 5   country                       10320 non-null  object        
 6   managed_by                    10320 non-null  object        
 7   fulfill_via                   10320 non-null  object        
 8   vendor_inco_term              10320 non-null  object        
 9   shipment_mode                 9960 non-null   object        
 10  pq_first_sent_to_client_date  4591 non-null   datetime64[ns]
 11  po_sent_to_vendor_date      

In [100]:
data['dosage']

0                  NaN
1              10mg/ml
2                  NaN
3                150mg
4                 30mg
             ...      
10315       30/50/60mg
10316        150/300mg
10317    600/300/300mg
10318        150/300mg
10319          30/60mg
Name: dosage, Length: 10320, dtype: object

manufacturers:
Wadhwan, IN




vendors: 
Larserpoortweg, NL

Remove ltd, llc, L.C., limited
We do not need to include for our purposes

After sorting alphabetically we see there are no duplicates.

There are many different manufacturing sites located in different locations in order to distribute commodities across multiple areas. For example, Abbvie is an American pharm company, previously part of Abbott. There are multiple manufacturing facilities in Japan, UK, Germany (Wiesbaden, Ludwigshafen) and in the U.S. (St. Petersburg, North Chicago). Thus, we will split manufacturing_site into 2 columns, manufacturer and manufacturing_site. The manufacturer will be, for example, abbvie (abbott). manufacturing_site will be the location, like 'france', 'north chicago, us'. https://www.abbviecontractmfg.com/locations.html

'Emcure Plot No.P-2, I.T-B.T. Park, Phase II, MIDC, Hinjwadi, Pune, India', is a typo that includes the address, should just be renamed 'Emcure Pharmaceuticals Limited, India'

'Weifa A.S., Hausmanngt. 6, P.O. Box 9113 GrÃ¸nland, 0133, Oslo, Norway'

BI, Ingelheim, Germany is the same as Boehringer Ingelheim

Guilin Pharmaceutical Co., Ltd. of Guilin, Guangxi.

In [ ]:
# convert to all Abbvie (Abbott) in manufacturer col
# convert all manufacturing_site 

In [ ]:
data.groupby(['vendor', 'manufacturing_site']).ngroups

In [ ]:
data.columns[17]

In [ ]:
# sort alphabetically to view
vend = data.sort_values('vendor')
vend['vendor'].unique()

In [ ]:
'''
#shipment_mode has 360 null values
sm_index = data.index[data['shipment_mode'] == 'no_value'].tolist()
print("Shipment Mode indexes:         ", len(shipment_indexes))
data = data.drop(sm_index)
'''